본 자료는 다음 링크의 내용을 참고하였습니다.

- Reference : https://www.kaggle.com/code/ibtesama/getting-started-with-a-movie-recommendation-system
- TMDB 5000 에서 수많은 영화 추천 게시글중 인기있는 예시를 가져온것이다.
- TMDB 5000은 TMDB에 영화 5000개를 데이터로 사용할 수 있게 데이터 셋 해둔 사이트이다.
- 출처 : [유튜버 나도코딩](https://www.youtube.com/watch?v=TNcfJHajqJY) 영상 참고

# 영화 추천 시스템

1. Demographic Filtering (인구통계학적 필터링)
1. Content Based Filtering (컨텐츠 기반 필터링)
1. Collaborative Filtering (협업 필터링)

이 중에서 컨텐츠 기반 필터링 방식 사용 => 줄거리 기반으로 사용

## 1. Dataset

In [ ]:
# drive mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Flutter Toy Project/dataset/tmdb_5000_movies.csv')

## 2. Content Based Filtering (컨텐츠 기반 필터링)
컨텐츠의 문자들을 유사도를 통해서 유사도가 상위인 데이터들 가져오는 형식

* `CountVectorizer` 미사용
  * 단어의 빈도수를 단순 카운트
  * 불용어도 따로 제거 가능(사용자 마음)
* `TfidfVectorizer(TF-IDF 기반의 벡터화)` 사용
  * 단어의 빈도수를 단순 카운트
  * 불용어 제거가 자동으로 라이브러리에 내장
    * 이는 자동으로 많은 불용어를 제거
  * 즉, 많은 문서 데이터가 있을경우 이 방식이 탁월하다는 것

### 줄거리 기반 추천
'overview'

In [ ]:
df['overview'].head(5)

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: overview, dtype: object

`TfidfVectorizer (TF-IDF 기반의 벡터화)`은 a, the 등등 어디 문서에서든 많이 나오므로 필요없는 이 영어들은 제외하고 나머지에서 위처럼 단어들을 필터링 해주는 방식을 의미

영화 줄거리는 이런 영어들 많이 사용하므로 이 기술을 사용

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

tfidf = TfidfVectorizer(stop_words='english') # 필요없는 영어들 제외
# ENGLISH_STOP_WORDS # 무엇을 제외했는지 보는 방법

DataFrame에 NaN 데이터 있는지 확인 및 '' 변환
* `pandas.DataFrame.isnull().values` 로 확인
  * 데이터 프레임의 NumPy 표현을 반환(T/F로)
* `numpy.any()` 로 요소 중 하나라도 `True` 라면 `True` 를 반환
* `fillna()` 로 DataFrame에 NaN(결측치) 를 찾아서 값을 바꿔줌

In [ ]:
# null 이 하나라도 있다면 true 반환
# for i in df['overview'].isnull().values:
#   if(i == True):
#     print("True")
#     break

df['overview'].isnull().values.any() # 위 주석을 한줄로 표현한 코드

# null 값을 찾아서 '' 값으로 삽입
df['overview'] = df['overview'].fillna('')

In [ ]:
# 모든 영화들 줄거리 내용을 띄어쓰기로 각각 단어로 분리했을때 20978개(중복포함)
tfidf_matrix = tfidf.fit_transform(df['overview'])
tfidf_matrix.shape # 20978개의 단어들이 모여 4803개의 문장이 된 것

(4803, 20978)

행렬과 행렬을 넣어주어 서로의 유사도를 계산
* 코사인 유사도는 얼마나 유사한 방향의 벡터를 가지는지의 정도

In [ ]:
# 신뢰도 - 코사인 유사도
from sklearn.metrics.pairwise import linear_kernel

cosine_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_matrix # 대각선은 같은 데이터니까 1 출력

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.02160533, 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.01488159, 0.        ,
        0.        ],
       ...,
       [0.        , 0.02160533, 0.01488159, ..., 1.        , 0.01609091,
        0.00701914],
       [0.        , 0.        , 0.        , ..., 0.01609091, 1.        ,
        0.01171696],
       [0.        , 0.        , 0.        , ..., 0.00701914, 0.01171696,
        1.        ]])

In [ ]:
# 크기
cosine_matrix.shape # 대칭

(4803, 4803)

In [ ]:
# 참고 : Series는 1차원 배열로 생각
indices = pd.Series(df.index, index=df['title']).drop_duplicates()
indices # 제목만 따로 구해둔 것

title
Avatar                                         0
Pirates of the Caribbean: At World's End       1
Spectre                                        2
The Dark Knight Rises                          3
John Carter                                    4
                                            ... 
El Mariachi                                 4798
Newlyweds                                   4799
Signed, Sealed, Delivered                   4800
Shanghai Calling                            4801
My Date with Drew                           4802
Length: 4803, dtype: int64

`cosine_matrix` 를 구했기 때문에 이제 활용만 하면 된다.
* 따라서 영화 추천받는 함수를 제작하자.
  * 입력 : 영화제목
  * 출력 : `cosine_matrix` 로 구한 추천 영화 5개

In [ ]:
def get_recommendations(title, cosine_sim=cosine_matrix):
    index = indices[title] # 입력 받는 영화의 index 구하기
    
    # 코사인 유사도 매트릭스 (cosine_sim) 에서 idx 에 해당하는 데이터를 (idx, 유사도) 형태로 얻기
    sim_scores = list(enumerate(cosine_sim[index])) # enumerate는 열거
    
    # 코사인 유사도 기준으로 내림차순 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 자기 자신을 제외한 5개의 추천 영화를 슬라이싱
    sim_scores = sim_scores[1:6]
    
    # 추천 영화 목록 5개의 인덱스 정보 추출
    movie_indices = [i[0] for i in sim_scores]
    
    # 인덱스 정보를 통해 영화 제목 추출
    return df['title'].iloc[movie_indices]

In [ ]:
# 람다식 사용방식 보여주기위함
def get_second(x):
    return x[1]

lst = ['인덱스', '유사도']
print(get_second(lst))

유사도


In [ ]:
# 람다식 사용방식 보여주기위함
# x[1]로 함수를 만든거고 lst가 x로 사용된거라고 생각하면 됨
(lambda x: x[1])(lst)

# 따라서 위에서 정렬할 때 유사도 기준으로 정렬하기 위해 간단히 람다식을 적용한 것

'유사도'

In [ ]:
# 함수 test
get_recommendations('Avengers: Age of Ultron')

16                   The Avengers
79                     Iron Man 2
68                       Iron Man
26     Captain America: Civil War
227                Knight and Day
Name: title, dtype: object

## 결과

찜목록에 영화 4개 있다고 가정하고, 4개 영화로 추천된 영화들 총 20개 영화 데이터들을 얻게 될것이며 이를 Flutter에 바로 넣겠다.

* 현재 데이터를 옛날 거로 사용했기 때문에, 찜목록 4개도 임의로 지정
  * `Avengers: Age of Ultron`
  * `Batman`
  * `Avatar`
  * `The Avengers`

In [ ]:
print(get_recommendations('Avengers: Age of Ultron'))
print()
print(get_recommendations('Iron Man'))
print()
print(get_recommendations('Avatar'))
print()
print(get_recommendations('The Avengers'))

16                   The Avengers
79                     Iron Man 2
68                       Iron Man
26     Captain America: Civil War
227                Knight and Day
Name: title, dtype: object

79                   Iron Man 2
31                   Iron Man 3
1868         Cradle 2 the Grave
7       Avengers: Age of Ultron
538                     Hostage
Name: title, dtype: object

3604               Apollo 18
2130            The American
634               The Matrix
1341    The Inhabited Island
529         Tears of the Sun
Name: title, dtype: object

7       Avengers: Age of Ultron
3144                    Plastic
1715                    Timecop
4124         This Thing of Ours
3311      Thank You for Smoking
Name: title, dtype: object
